In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 8s - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160 - 8s/epoch - 5ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7656 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8066 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.5553 - accuracy: 0.8157 - val_loss: 0.5350 - val_accuracy: 0.8238 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.5338 - accuracy: 0.8224 - val_loss: 0.5157 - val_accuracy: 0.8304 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.5172 - accuracy: 0.8272 - val_loss: 0.5079 - val_accuracy: 0.8282 - 5s/epoch - 3ms/step
Epoch 9/

# Batch Normalization

#### Internal Covariate Shift
* We define Internal Covariate Shift as the change in the
distribution of network activations due to the change in
network parameters during training. 

* To improve the training, we seek to reduce the internal covariate shift. By
fixing the distribution of the layer inputs x as the training
progresses, we expect to improve the training speed. 

* It has been long known (LeCun et al., 1998b; Wiesler & Ney,
2011) that the network training converges faster if its inputs are whitened – i.e., linearly transformed to have zero
means and unit variances, and decorrelated. 

* As each layer observes the inputs produced by the layers below, it would
be advantageous to achieve the same whitening of the inputs of each layer. 

* By whitening the inputs to each layer, we would take a step towards achieving the fixed distributions of inputs that would remove the ill effects of the
internal covariate shift.

reference [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/pdf/1502.03167.pdf)

## Input: 
### Values of x over a mini-batch: $B = \{x_{1...m}\}$
### Learnable parameters: $\gamma$ and $\beta$


## Output: 
### $\{z^{(i)} = BN _{\gamma, \beta}(x^{(i)})\}$

## Algorithm:

### 1. mini-batch mean: $\mu_B = \frac{1}{m_B} \sum_{i=1}^{m_B} x^{(i)}$

### 2. mini-batch variance: $\sigma_B^2 = \frac{1}{m_B} \sum_{i=1}^{m_B} (x^{(i)} - \mu_B)^2$

### 3. normalize: $\hat{x}^{(i)} = \frac{x^{(i)} - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$

### 4. scale and shift: $ z^{(i)} = \gamma \otimes  \hat{x}^{(i)} + \beta \equiv BN _{\gamma, \beta}(x^{(i)})\ $ 

---

## BN Approach One

In [7]:
del model

In [8]:
LAYERS_BN = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN)

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [10]:
784 * 4 , 300 * 4 , 100 * 4

784 * 4 + 300 * 4 + 100 * 4

(784 * 4 + 300 * 4 + 100 * 4)/2

2368.0

In [11]:
784 * 4 # mean, variance, gamma and Beta

3136

In [12]:
300 * 4

1200

In [13]:
100 *4 

400

In [14]:
3136 + 1200 + 400

4736

In [15]:
4736 / 2

2368.0

In [16]:
266610 + 2368.0

268978.0

In [17]:
266610 + 4736

271346

In [18]:
bn1 = model.layers[1]
for variable in bn1.variables:
    print(f"variable name: {variable.name.split('/')[-1][:-2]}, \nis trainable: {variable.trainable}\n\n")

variable name: gamma, 
is trainable: True


variable name: beta, 
is trainable: True


variable name: moving_mean, 
is trainable: False


variable name: moving_variance, 
is trainable: False




In [19]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [20]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 11s - loss: 0.8294 - accuracy: 0.7221 - val_loss: 0.5540 - val_accuracy: 0.8156 - 11s/epoch - 6ms/step
Epoch 2/10
1719/1719 - 9s - loss: 0.5703 - accuracy: 0.8035 - val_loss: 0.4792 - val_accuracy: 0.8380 - 9s/epoch - 5ms/step
Epoch 3/10
1719/1719 - 8s - loss: 0.5161 - accuracy: 0.8213 - val_loss: 0.4424 - val_accuracy: 0.8494 - 8s/epoch - 5ms/step
Epoch 4/10
1719/1719 - 9s - loss: 0.4789 - accuracy: 0.8314 - val_loss: 0.4213 - val_accuracy: 0.8558 - 9s/epoch - 5ms/step
Epoch 5/10
1719/1719 - 9s - loss: 0.4548 - accuracy: 0.8407 - val_loss: 0.4051 - val_accuracy: 0.8608 - 9s/epoch - 5ms/step
Epoch 6/10
1719/1719 - 9s - loss: 0.4387 - accuracy: 0.8444 - val_loss: 0.3930 - val_accuracy: 0.8638 - 9s/epoch - 5ms/step
Epoch 7/10
1719/1719 - 9s - loss: 0.4255 - accuracy: 0.8503 - val_loss: 0.3830 - val_accuracy: 0.8632 - 9s/epoch - 5ms/step
Epoch 8/10
1719/1719 - 9s - loss: 0.4123 - accuracy: 0.8539 - val_loss: 0.3758 - val_accuracy: 0.8666 - 9s/epoch - 5ms/step
Epoch 

## BN Approach Two

Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [21]:
del model

In [22]:
LAYERS_BN_BIAS_FALSE = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN_BIAS_FALSE)

In [23]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [24]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 12s 6ms/step - loss: 1.0346 - accuracy: 0.6739 - val_loss: 0.6680 - val_accuracy: 0.7884
Epoch 2/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6757 - accuracy: 0.7819 - val_loss: 0.5537 - val_accuracy: 0.8212
Epoch 3/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5961 - accuracy: 0.8021 - val_loss: 0.4996 - val_accuracy: 0.8352
Epoch 4/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5456 - accuracy: 0.8176 - val_loss: 0.4655 - val_accuracy: 0.8458
Epoch 5/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5140 - accuracy: 0.8250 - val_loss: 0.4419 - val_accuracy: 0.8510
Epoch 6/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4915 - accuracy: 0.8309 - val_loss: 0.4237 - val_accuracy: 0.8540
Epoch 7/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4741 - accuracy: 0.8375 - val_loss: 0.4103 - val_ac